In [ ]:
# edit code so that it runs automatically every hour
# use python.12.4 kernel

In [ ]:
import contextily as ctx


In [ ]:
import requests
import geopandas as gpd

# Construct the query URL
query_url = "https://maps.water.noaa.gov/server/rest/services/nwm/srf_18hr_max_high_flow_magnitude/MapServer/0/query"
params = {
    'where': '1=1',
    'outFields': '*',
    'outSR': '4326',  # Specify output spatial reference if needed
    'f': 'geojson',  # Request geojson output
    'returnGeometry': 'true'
}

# Make the request
response = requests.get(query_url, params=params)

# Convert to GeoDataFrame
predictions = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")

print(predictions.head())


In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import contextily as ctx

# Create a subplot with specified figure size
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the predictions GeoDataFrame on the created axes object
# Ensure to specify the ax parameter to plot on the same axes
predictions.plot(ax=ax, color='blue', edgecolor='black')

# Add a basemap to the same axes object
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

# Adjust x and y axis labels to longitude and latitude
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Convert axis labels to degrees and set rotation for better readability
ax.set_xticklabels([round(val, 2) for val in ax.get_xticks()], rotation=45)
ax.set_yticklabels([round(val, 2) for val in ax.get_yticks()])

# Set plot title and labels for axes
plt.title('NOAA Water Maps - 5 Day Max Inundation Extent')

# Display the combined plot
plt.show()

In [ ]:
# Export GeoDataFrame as a Shapefile
predictions.to_file("C:/Users/Lyn/Documents/Programming/Python_codes/CUAHSI Summer Institute 2024/18hr_highflow_test.shp")

Import usgs current streamflow data to combine with the flood discharge predictions

In [10]:
# import from the website and save as a csv (comma seperated) in x location

In [ ]:
# read csv
import pandas as pd

# Replace 'path/to/your_file.csv' with the actual file path
usgs = pd.read_csv('C:/Users/Lyn/Documents/Homework/UMass/CUAHSI_2024/forecastdata/streamflow_texas_06252024.csv')

print(df.head())

# Remove the header
usgs.drop(index=range(0, 29), inplace=True)  # Removes the header
print(df.head())

If there are empty values in our area of interest, we need to fill them. We will first fill them with usgs data if it exists. the remaining data will be filled with 0s. 

In [ ]:
import pandas as pd

# Import the file with the feature ids we need to fill in our huc of interest.
featureIDs = pd.read_csv('C:/Users/Lyn/Documents/Homework/UMass/CUAHSI_2024/FIM_Harvey/featureID_Houston.csv')
featureIDs['feature_id'] = featureIDs['feature_id'].astype('int64')

# extract the flow data from the NWM streamflow forecast data
maxflowpredictions = predictions[['feature_id', 'max_flow']]
# Ensure 'feature_id' in maxflowpredictions is of type int64
maxflowpredictions['feature_id'] = maxflowpredictions['feature_id'].astype('int64')

# Merge featureIDs with maxflowpredictions based on the 'feature_id' column
flowpredictions = pd.merge(featureIDs, maxflowpredictions, on='feature_id', how='left')
flowpredictions.fillna(0, inplace=True) # fill all missing values with 0
print(flowpredictions)


transpose the data into a format that can be fed into handfim

In [ ]:

transposed_columns = maxflowpredictions.T

# Specify the datetime string you want to use
new_datetime = '2024-06-25 12:50:32 EDT'

# Rename 'max_flow' to the specified datetime
forecastinput = transposed_columns.rename(index={'max_flow': new_datetime})
print(forecastinput.head())

## export the data to a csv file with the date and time in the filename
# Replace invalid filename characters in new_datetime
filename_datetime = new_datetime.replace(' ', '_').replace(':', '_')
# Format the filename string to include new_datetime
filename = f'forecast_input_{filename_datetime}.csv'
print(filename)
# Export forecastinput as a CSV file with the formatted name
forecastinput.to_csv(filename, sep=',', index=True)
